In [ ]:
import pandas as pd
import numpy as np
import nltk


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")


In [ ]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


In [ ]:
# df.loc[i,'tweet'].lower()
# df.tweet=df.tweet.apply(lambda x : x.lower())
df.tweet=df.tweet.str.lower()

# df.loc[0,'tweet'].lower()

In [ ]:
df

,id,tweet,label
0,1,the cdc currently reports 99031 deaths. in gen...,real
1,2,states reported 1121 deaths a small rise from ...,real
2,3,politically correct woman (almost) uses pandem...,fake
3,4,#indiafightscorona: we have 1524 #covid testin...,real
4,5,populous states can generate large case counts...,real
...,...,...,...
6415,6416,a tiger tested positive for covid-19 please st...,fake
6416,6417,???autopsies prove that covid-19 is??� a blood...,fake
6417,6418,_a post claims a covid-19 vaccine has already ...,fake
6418,6419,aamir khan donate 250 cr. in pm relief cares fund,fake


In [ ]:
# In order to remove any URL within a string in Python, you can use this RegEx function :

import re 
for i in range(df.shape[0]):
    df.loc[i,'tweet']=re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", df.loc[i,'tweet'])
df.loc[4,'tweet']

'populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than california or texas: al ar id ks ky la ms nv and sc.  '

In [ ]:
df.loc[4,'tweet']


'populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than california or texas: al ar id ks ky la ms nv and sc.  '

We try to remove emojis

In [ ]:
#I took an example to work on 
example = df.loc[6416,'tweet']
example

'???autopsies prove that covid-19 is??� a blood clot, not pneumonia, ???and ought to be fought with antibiotics??� and the whole world has been wrong in treating the ???so-called??� pandemic.'

In [ ]:
#first we need to turn text into pure characters meaning the emojis must be converted to their string form
#to do that we use the function encode
example.encode('utf-8')

b'???autopsies prove that covid-19 is??\xef\xbf\xbd a blood clot, not pneumonia, ???and ought to be fought with antibiotics??\xef\xbf\xbd and the whole world has been wrong in treating the ???so-called??\xef\xbf\xbd pandemic.'

In [ ]:
example

'???autopsies prove that covid-19 is??� a blood clot, not pneumonia, ???and ought to be fought with antibiotics??� and the whole world has been wrong in treating the ???so-called??� pandemic.'

In [ ]:
# Remove all traces of emoji from a text file.

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
remove_emoji(df.loc[6416,'tweet'])


'???autopsies prove that covid-19 is?? a blood clot, not pneumonia, ???and ought to be fought with antibiotics?? and the whole world has been wrong in treating the ???so-called?? pandemic.'

In [ ]:
for i in range(df.shape[0]):
  df.loc[i,'tweet']=remove_emoji(df.loc[i,'tweet'])
  


In [ ]:
df

,id,tweet,label
0,1,the cdc currently reports 99031 deaths. in gen...,real
1,2,states reported 1121 deaths a small rise from ...,real
2,3,politically correct woman (almost) uses pandem...,fake
3,4,#indiafightscorona: we have 1524 #covid testin...,real
4,5,populous states can generate large case counts...,real
...,...,...,...
6415,6416,a tiger tested positive for covid-19 please st...,fake
6416,6417,???autopsies prove that covid-19 is?? a blood ...,fake
6417,6418,_a post claims a covid-19 vaccine has already ...,fake
6418,6419,aamir khan donate 250 cr. in pm relief cares fund,fake


In [ ]:
We try to remove emojis

In [ ]:
#i took an example to work on 
example = df.loc[89,'tweet']
example

In [ ]:
#the easiet solution we found is to turn text incode into ascii wich does not include emojis and ignore all other characters that are not defined on ascii encode
#we use encode function which turns string into ascii code and ignoring none ascii characters
#then decode to make the ascii code a string again
(example.encode('ascii',errors='ignore')).decode('ascii')

In [ ]:
toAscii = lambda text : (text.encode('ascii',errors='ignore')).decode('ascii')
df.tweet = df.tweet.apply(toAscii)

In [ ]:
df

In [ ]:
df.loc[89].tweet